# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [5]:
from pprint import pprint
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [6]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [9]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["../data/metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [11]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [12]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [13]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [14]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [15]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [16]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that utilizes human-like Standardized Operating Procedures (SOPs) to enhance multi-agent collaborations based on Large Language Models (LLMs). MetaGPT assigns specific roles to agents, streamlines workflows, and improves communication efficiency. It incorporates structured outputs, role specialization, efficient sharing mechanisms, and an executable feedback mechanism for self-correction during runtime. MetaGPT achieves state-of-the-art performance in code generation benchmarks, showcasing its success in enhancing collaboration and code generation quality in artificial multi-agent systems. The framework's focus on communication efficiency, role specialization, and iterative programming with executable feedback highlights its potential for improving problem-solving processes and code quality in software development scenarios.


In [20]:
print(len(response.source_nodes))

34


In [21]:
pprint(response.source_nodes)

[NodeWithScore(node=TextNode(id_='71c527fe-7fdb-41ae-8c71-5e8f896a2513', embedding=None, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': '../data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eedd8546-5ebf-4d5f-bcad-48a9e34500db', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': '../data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}, hash='0eb1c97c692f8437f4c78a903534532cdeeb85f9f77fe6931e41a56a56f9f150

In [23]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
pprint(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context, which is necessary for understanding how agents share information with other agents..
('Agents share information with other agents by utilizing a shared message '
 'pool where they can publish structured messages. This shared message pool '
 'allows all agents to exchange messages directly, enabling them to both '
 'publish their own messages and access messages from other agents '
 'transparently. Agents can retrieve required information directly from this '
 'shared pool, eliminating the need to inquire about other agents and wait for '
 'their responses, thus enhancing communication efficiency.')


## Let's put everything together

In [26]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("../data/metagpt.pdf")

In [30]:
response = query_engine.query("Summarize the conclusions.")

Selecting query engine 0: The conclusion of the paper is likely related to summarization questions, making choice 1 the most relevant..


In [31]:
pprint(str(response))

('The conclusions drawn from the provided information highlight the focus on '
 'self-improvement mechanisms in software development teams through active '
 'teamwork and recursive self-improvement. The concept of learning from '
 'experience and meta-learning better algorithms is emphasized, with recent '
 'work leveraging Large Language Models (LLMs) to enhance performance on '
 'tasks. Additionally, the exploration of self-referential mechanisms that '
 'modify constraint prompts based on project feedback is discussed. The idea '
 'of multi-agent economies, where agents contribute to economic success in a '
 'Reinforcement Learning framework, is presented. The platform AgentStore '
 'facilitates collaboration among agents within the MetaGPT framework, '
 'allowing users to manage agents with different capabilities for specific '
 'tasks. Overall, the emphasis is on continuous learning, adaptation, and '
 'collaboration to improve software development processes.')
